In [163]:
#pip install supabase


     ---------------------------------------- 48.4/48.4 kB 1.2 MB/s eta 0:00:00
  Using cached httpx-0.23.3-py3-none-any.whl (71 kB)
     ---------------------------------------- 55.1/55.1 kB ? eta 0:00:00
     -------------------------------------- 216.4/216.4 kB 2.2 MB/s eta 0:00:00
     -------------------------------------- 184.3/184.3 kB 2.8 MB/s eta 0:00:00
     -------------------------------------- 135.8/135.8 kB 4.1 MB/s eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 5.9 MB/s eta 0:00:00
  Using cached httpcore-0.16.3-py3-none-any.whl (69 kB)
  Using cached rfc3986-1.5.0-py2.py3-none-any.whl (31 kB)
     -------------------------------------- 101.4/101.4 kB 5.7 MB/s eta 0:00:00
     ---------------------------------------- 62.7/62.7 kB 1.7 MB/s eta 0:00:00
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
     ---------------------------------------- 54.5/54.5 kB 2.8 MB/s eta 0:00:00


In [258]:
import requests
from datetime import date
import datetime
from supabase import create_client, Client

In [265]:
#Supabase connection strings
url="o"
key=""

supabase_conn= create_client(url, key)

In [252]:
import logging
logger = logging.getLogger()
loghandler = logging.FileHandler(filename='StockAnalysis_log.log', mode='a')
logger.addHandler(loghandler)


In [253]:
#logging.info('Logs for Stock Analysis ETL Process')

2023-05-26 21:36:38,501:INFO - Logs for Stock Analysis ETL Process


In [261]:
today = date.today()
#today=today-datetime.timedelta(days=11)
print("Today's date:", today)

Today's date: 2023-05-27


In [262]:

url_day = "https://working-days.p.rapidapi.com/1.3/get_info_day"

querystring = {"country_code":"US","date":today-datetime.timedelta(days=1),"configuration":"Federal holidays"}

headers = {
	"X-RapidAPI-Key": "",
	"X-RapidAPI-Host": "working-days.p.rapidapi.com"
}

response = requests.request("GET", url_day, headers=headers, params=querystring)

print(response.text)

{
"working_day":1,
"work_hours":8,
"wages":160,
"morning_start":"08:00",
"morning_end":"12:00",
"afternoon_start":"14:00",
"afternoon_end":"18:00",
"public_holiday":"0",
"public_holiday_description":"",
"weekend_day":0,
"custom_date":0,
"custom_date_description":"",
"custom_date_color":"",
"type":32
}


In [266]:
today = date.today()
yesterday=today-datetime.timedelta(days=1)
print(d, "Yesterday's date:", yesterday)

if (response.json()['working_day']==1) :

    logging.info('Extract Operation for'+str(yesterday))

    url='https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=IBM&apikey=ODSLP5A1Z79WG98U&outputsize=compact'
    r = requests.get(url)
    data = r.json()

    #print(data)

    SMA_nValue= 3
    LMA_nValue=10
    leaveDays=0
    SMA=0
    LMA=0
    Sum=0
    i=0;
    
    while(LMA_nValue+leaveDays>i):
        if ( (yesterday-datetime.timedelta(days=i)).strftime('%Y-%m-%d')) in data['Time Series (Daily)'] :
            #print( (yesterday-datetime.timedelta(days=i)).strftime('%Y-%m-%d') ," -- ",data['Time Series (Daily)'][ (yesterday-datetime.timedelta(days=i) ).strftime('%Y-%m-%d')]['4. close'],"--working")
            Sum+=float(data['Time Series (Daily)'][ (yesterday-datetime.timedelta(days=i) ).strftime('%Y-%m-%d')]['4. close'])
            if i == SMA_nValue+leaveDays-1:
                SMA='%.3f'% (Sum/SMA_nValue)

            if i == LMA_nValue+leaveDays-1:
                LMA='%.3f'% (Sum/LMA_nValue)
        else:
            #print(i,"-- Leave")
            leaveDays+=1;
        i+=1;

    print("SMA -:", SMA,"LMA -:", LMA)
    logging.info('Transformted Data:'+"SMA -"+str(SMA)+"LMA -"+str(LMA))

    value={'Date':(yesterday).strftime('%Y-%m-%d'),'SMA':SMA,'LMA':LMA}
    op=supabase_conn.table('StockAnalysis').insert([value]).execute()
    logging.info('Load operation output:'+str(op))
    print(op)

else:
    logging.info('Holiday -'+str(yesterday))

2023-05-27 10:19:46,336:INFO - Extract Operation for2023-05-26


1 Yesterday's date: 2023-05-26


2023-05-27 10:19:48,250:INFO - Transformted Data:SMA -127.110LMA -126.295


SMA -: 127.110 LMA -: 126.295


2023-05-27 10:19:49,663:INFO - Load operation output:data=[{'Date': '2023-05-26', 'SMA': 127.11, 'LMA': 126.295, 'created_at': '2023-05-27T04:49:50.296655+00:00'}] count=None


data=[{'Date': '2023-05-26', 'SMA': 127.11, 'LMA': 126.295, 'created_at': '2023-05-27T04:49:50.296655+00:00'}] count=None
